# Libraries

In [1]:
import numpy as np
import time

from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *
from pkg.algorithims  import local_search

# VNS

## Parámetros del VNS

In [11]:
# semillas usadas para la experimentacion
seeds = np.array([7054, 1354, 23503, 11268, 58283])

# Número máximo de entornos explorados
k_max = 4

# Tamaños de los entornos (en orden creciente)
sizes_of_sublists = [4, 8, 12, 16]

# Granularidad del movimiento, calculada previamente
granularity = 2 

# Numero de intentos (mutaciones fuertes) al llegar al ultimo entorno
attempts = 5

In [3]:
ev    = Evaluation()

In [6]:
# Calculamos el tiempo medio de ejecución de 1 evaluación

t_total = 0

for i in range(100):
    rs = generate_random_solution()

    ini = time.time()
    ev.evaluate(rs)
    fin = time.time()

    t_total += (fin-ini)

t_medio = t_total/100
print(t_medio) 

0.05094067573547363


Calculamos el parámetro **bl_max**, que indica el criterio de salida. Para calcularlo se tienen en cuenta los siguientes datos:
- Número de evaluaciones medias por BL: 892.6 evaluaciones (dato obtenido de la práctica anterior)
- Tiempo de ejecución de 1 evaluación: 0.05094067573547363 segundos

Por tanto, el tiempo de ejecución medio de una BL es de 45 segundos aproximadamente.

El valor de **bl_max** se establece de tal forma que el tiempo de ejecución de una búsqueda VNS no supere los 30 minutos. Como una BL tarda de media menos de 1 minuto, podemos establecer que bl_max = 30. De esta forma, aunque nos encontremos con una BL con una duración mayor a la media, el tiempo aproximado de la búsqueda VNS seguirá rondando los 30 minutos.

In [4]:
bl_max = 30

## Operador de generador de vecino

Comprobamos el correcto funcionamiento del operador de generador de vecino

In [5]:
ks = list(range(1,k_max+1))

for k in ks:

    solution = generate_random_solution()

    neighbor = neighbor_generation_operator(
        solution    = solution, 
        k           = k,
        sizes_Ek    = sizes_of_sublists,
        granularity = granularity
    )

    print(f'\nk = {k}')
    print(solution)
    print(neighbor)


k = 1
[ 9  6  9 11 14  9 24  6 23 14 17 17 23  9 20  9]
[11  4 11 11 14  9 24  6 23 14 17 17 23  9 20  7]

k = 2
[ 8  8  8  6 19 25 11  8 17 11  6 25  8 22 14 22]
[ 8  8  8  6 19 23 13  6 19  9  8 23 10 22 14 22]

k = 3
[19  6 11  6  6 19 22  8 23 22  8 19 14  6 25  6]
[19  6 11  6  4 21 20 10 21 24  6 21 12  8 23  8]

k = 4
[12 12  6 15 21 15 21  9 21  6 18 12  6 24  6 12]
[10 14  4 17 19 17 19 11 19  8 16 14  4 26  4 14]


## Algoritmo

In [8]:
def vns(solution, k_max, bl_max, sizes_Ek, ev, path=None):

    """
    Implementación del algoritmo VNS


    Parametros
    ----------
    k_max: Numero máximo de entornos a explorar.

    bl_max: Máximo número de BL a realizar. Criterio de parada

    solution: Solución inicial

    sizes_Ek: Tamaño de los entornos

    ev: Objeto evaluación. Aplica la función coste sobre una solución.

    path: Parametro opcional. Nombre y ruta del archivo que almacena los datos 
    de experimentación.


    Returns
    -------
    global_sol: Array de enteros que representa la capacidad de las estaciones.

    global_sol_cost: COste de global_sol
    """

    # conteo de evaluaciones realizadas
    ev.total_calls = 0

    
    global_sol      = solution
    global_sol_cost = ev.evaluate(global_sol)


    k  = 1
    bl = 0

    while k <= k_max and bl <= bl_max :

        neighbor = neighbor_generation_operator(
            solution    = global_sol, 
            k           = k,
            sizes_Ek    = sizes_Ek,
            granularity = granularity
        )

        candidate_sol, candidate_sol_cost = local_search(
            granularity = granularity,
            evaluation  = ev,
            initial_sol = neighbor
        )

        bl += 1
        print(bl)

        if candidate_sol_cost < global_sol_cost:
            print(f'mejora')
            print(f'Coste antiguo: {global_sol_cost}')
            print(f'Coste nuevo: {candidate_sol_cost}')
            global_sol      = candidate_sol
            global_sol_cost = candidate_sol_cost

            k = 1
        else:
            print('no mejora')
            k += 1

    return global_sol, global_sol_cost

In [9]:
for s in seeds:
    np.random.seed(s)

    solution = generate_random_solution()
    sol, cost = vns(
        solution = solution,
        k_max = k_max,
        bl_max = bl_max,
        sizes_Ek= sizes_of_sublists,
        ev=ev
    )
    

1
mejora
Coste antiguo: 564.9951766145007
Coste nuevo: 375.7658566586277
2
no mejora
3
no mejora
4
no mejora
5
no mejora
1
mejora
Coste antiguo: 522.6903616556751
Coste nuevo: 379.4289296151759
2
no mejora
3
mejora
Coste antiguo: 379.4289296151759
Coste nuevo: 377.0153450666003
4
no mejora
5
no mejora
6
no mejora
7
no mejora
1
mejora
Coste antiguo: 570.9421348371571
Coste nuevo: 393.843538106775
2
no mejora
3
no mejora
4
no mejora
5
no mejora
1
mejora
Coste antiguo: 508.07413744520755
Coste nuevo: 377.2167786072197
2
no mejora
3
mejora
Coste antiguo: 377.2167786072197
Coste nuevo: 374.6904512366752
4
no mejora
5
no mejora
6
no mejora
7
mejora
Coste antiguo: 374.6904512366752
Coste nuevo: 371.85391617579
8
no mejora
9
no mejora
10
no mejora
11
no mejora
1
mejora
Coste antiguo: 494.99265776674383
Coste nuevo: 414.1047597059338
2
no mejora
3
mejora
Coste antiguo: 414.1047597059338
Coste nuevo: 413.2058938025359
4
mejora
Coste antiguo: 413.2058938025359
Coste nuevo: 409.6756059650655
5
no 

## Algoritmo mejorado

Debido a que el algoritmo se estanca antes de llegar al número máximo de búsquedas locales (bl_max = 30), incorporamos una mejora del algoritmo. De esta forma, en una última instancia, se incorpora el parámetro "intentos", que realiza 5 mutaciones fuertes (fuertes porque el entorno es k=4), dado que cada mutación arroja un resultado distinto.

In [10]:
def vns_upgrade(solution, k_max, bl_max, attempts, sizes_Ek, ev, path=None):

    """
    Implementación del algoritmo VNS


    Parametros
    ----------
    solution: Solución inicial

    k_max: Numero máximo de entornos a explorar.

    bl_max: Máximo número de BL a realizar. Criterio de parada

    attempts: Numero máximo de intentos en el último entorno

    sizes_Ek: Tamaño de los entornos

    ev: Objeto evaluación. Aplica la función coste sobre una solución.

    path: Parametro opcional. Nombre y ruta del archivo que almacena los datos 
    de experimentación.


    Returns
    -------
    global_sol: Array de enteros que representa la capacidad de las estaciones.

    global_sol_cost: COste de global_sol
    """

    # conteo de evaluaciones realizadas
    ev.total_calls = 0

    
    global_sol      = solution
    global_sol_cost = ev.evaluate(global_sol)


    k  = 1
    bl = 0
    a  = 0 # numero de intentos

    while a < attempts and bl < bl_max :

        
        
        neighbor = neighbor_generation_operator(
            solution    = global_sol, 
            k           = k,
            sizes_Ek    = sizes_Ek,
            granularity = granularity
        )

        candidate_sol, candidate_sol_cost = local_search(
            granularity = granularity,
            evaluation  = ev,
            initial_sol = neighbor
        )

        bl += 1
        print(bl)

        if candidate_sol_cost < global_sol_cost:
            print(f'mejora')
            print(f'Coste antiguo: {global_sol_cost}')
            print(f'Coste nuevo: {candidate_sol_cost}')
            global_sol      = candidate_sol
            global_sol_cost = candidate_sol_cost

            k = 1
            a = 0
            
        else:
            print('no mejora')
            

            if k < k_max:
                k += 1
            else:
                a += 1
            

    return global_sol, global_sol_cost

In [12]:
for s in seeds:
    np.random.seed(s)

    solution = generate_random_solution()
    sol, cost = vns_upgrade(
        solution = solution,
        k_max = k_max,
        bl_max = bl_max,
        attempts= attempts,
        sizes_Ek= sizes_of_sublists,
        ev=ev
    )

1
mejora
Coste antiguo: 564.9951766145007
Coste nuevo: 375.7658566586277
2
no mejora
3
no mejora
4
no mejora
5
no mejora
6
no mejora
7
no mejora
8
no mejora
9
no mejora
1
mejora
Coste antiguo: 522.6903616556751
Coste nuevo: 379.4289296151759
2
no mejora
3
mejora
Coste antiguo: 379.4289296151759
Coste nuevo: 377.0153450666003
4
no mejora
5
no mejora
6
no mejora
7
no mejora
8
no mejora
9
mejora
Coste antiguo: 377.0153450666003
Coste nuevo: 376.0421355251909
10
no mejora
11
no mejora
12
no mejora
13
no mejora
14
no mejora
15
no mejora
16
no mejora
17
no mejora
1
mejora
Coste antiguo: 570.9421348371571
Coste nuevo: 393.843538106775
2
no mejora
3
no mejora
4
no mejora
5
no mejora
6
no mejora
7
no mejora
8
no mejora
9
no mejora
1
mejora
Coste antiguo: 508.07413744520755
Coste nuevo: 377.2167786072197
2
no mejora
3
mejora
Coste antiguo: 377.2167786072197
Coste nuevo: 374.6904512366752
4
no mejora
5
no mejora
6
no mejora
7
mejora
Coste antiguo: 374.6904512366752
Coste nuevo: 371.85391617579
8


No se ve mejoria ninguna en los resultados.

Posibles enfoques: aumentar el numero de intentos en el ultimo entorno.
                    Probar con nuevas semillas, igual las soluciones iniciales no son buenas.